Importing the libraries

In [12]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import math

Load the dataset

In [13]:
df = pd.read_csv("Social_Network_Ads.csv")

Data pre-processing

In [14]:
# Manual one-hot encoding
df['Gender'] = df['Gender'].map({'Male': 0, 'Female': 1})

# features used for predicting and the actual target variable
X = df[['Gender', 'Age', 'EstimatedSalary']]
y = df['Purchased']

Training and testing data splitting

In [15]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

Standardise the dataset - for better convergence in admm

In [16]:
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

Activation for regression - manual

In [17]:
def sigmoid(z):
    return 1 / (1 + np.exp(-z))

A loss function is defined

This is the function to be used in the ADMM

In [18]:
def logistic_loss(X, y, beta, lambda_):
    m = len(y)
    predictions = sigmoid(X @ beta)
    # the following is just a loss function formula
    loss = - (1 / m) * (y @ np.log(predictions + 1e-9) + (1 - y) @ np.log(1 - predictions + 1e-9)) + (lambda_ / 2) * np.sum(beta ** 2)
    # 1e-9 is added to the prediction to avoid log(0) numerical errors
    return loss

Functions


In [19]:
def dot_product(a, b):
    """Computes the dot product of two vectors."""
    return sum(x * y for x, y in zip(a, b))

def matrix_vector_mult(matrix, vector):
    """Multiplies a matrix with a vector."""
    return [dot_product(row, vector) for row in matrix]

def vector_add(a, b, scale=1.0):
    """Adds two vectors element-wise with an optional scaling factor."""
    return [x + scale * y for x, y in zip(a, b)]

def vector_subtract(a, b):
    """Subtracts vector b from vector a."""
    return [x - y for x, y in zip(a, b)]

def scalar_vector_mult(scalar, vector):
    """Multiplies each element in a vector by a scalar."""
    return [scalar * x for x in vector]

def l2_norm(vector):
    """Calculates the L2 norm of a vector."""
    return math.sqrt(sum(x ** 2 for x in vector))

def logistic_loss(X, y, beta, lambda_):
    """Calculates logistic loss with L2 regularization."""
    m = len(y)
    total_loss = 0
    for i in range(m):
        predicted = sigmoid(dot_product(X[i], beta))
        total_loss += -y[i] * math.log(predicted) - (1 - y[i]) * math.log(1 - predicted)
    regularization = (lambda_ / 2) * dot_product(beta, beta)
    return (total_loss / m) + regularization


ADMM Algorithm

In [20]:
def admm_logistic_regression(X, y, lambda_=1.0, rho=1.0, max_iter=1000, tol=1e-6):

    m = len(X)
    n = len(X[0])     


    beta = [0.01] * n
    z = [0] * n
    u = [0] * n


    for iteration in range(max_iter):
        gradient = [0] * n

        for i in range(m):
            error = sigmoid(dot_product(X[i], beta)) - y[i]
            
            for j in range(n):
                gradient[j] += X[i][j] * error
        
        for j in range(n):
            gradient[j] = (gradient[j] / m) + lambda_ * beta[j] + rho * (beta[j] - z[j] + u[j])

        beta_update = [(gradient[j]) / (rho + lambda_ + m) for j in range(n)]
        beta = vector_subtract(beta, beta_update)


        z_old = z.copy()
        z = vector_add(beta, u)


        u = vector_add(u, vector_subtract(beta, z))


        loss = logistic_loss(X, y, beta, lambda_)
        predictions = [sigmoid(dot_product(X[i], beta)) >= 0.5 for i in range(m)]
        accuracy = sum(pred == actual for pred, actual in zip(predictions, y)) / m

        
        primal_residual = l2_norm(vector_subtract(beta, z))
        dual_residual = l2_norm(scalar_vector_mult(-rho, vector_subtract(z, z_old)))
        if primal_residual < tol and dual_residual < tol:
            print(f'The model converged after {iteration + 1} epochs.')

            print(f'Final accuracy: {accuracy * 100:.2f}%')
            return beta, accuracy  

    print('The model did not converge within the maximum number of epochs.')
    return beta, None  


Model fitting

In [21]:
X_train = X_train.tolist() if not isinstance(X_train, list) else X_train
y_train = y_train.tolist() if not isinstance(y_train, list) else y_train


beta_admm, acc_val = admm_logistic_regression(X_train, y_train, 10.0, 1.0)
print('Optimized set of weights:', beta_admm)

The model converged after 207 epochs.
Final accuracy: 80.62%
Optimized set of weights: [np.float64(0.0014701764530638435), np.float64(0.027304884057911625), np.float64(0.017679981460369917)]
